### 加速度とジャークを考慮した停止距離計算

#### 必要なパラメータ
- 現在速度
- 終端速度
- 現在加速度
- 最小加速度
- 最大ジャーク
- 最小ジャーク

In [1]:
# import library

from IPython.display import Image, display_png
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, FloatSlider
import numpy as np

In [2]:
def trapezoid_shape(v0, a0, a_min, j_acc, j_dec, t_min):

    t1 = (a_min - a0) / j_dec
    if t1 < 0.01:
        t1 = 0.0
    print("t1: %f" % t1)

    t2 = t_min
    if t2 < 0.01:
        t2 = 0.0
    print("t2: %f" % t2)

    t3 = -a_min / j_acc
    if t3 < 0.01:
        t3 = 0.0
    print("t3: %f" % t3)

    dists = []
    vels = []
    accs = []
    jerks = []
    times = []

    # 0 ~ t1
    for t in np.linspace(0.0, t1, 100, endpoint=True):

        a = a0 + (j_dec * t)
        v = v0 + (a0 * t) + (0.5 * j_dec * t * t)
        x = (v0 * t) + (0.5 * a0 * t * t) + ((1.0 / 6.0) * j_dec * t * t * t)
        # print("t: %f, t_total: %f, x: %f, v: %f, a: %f" % (t, t, x, v, a))

        dists.append(x)
        vels.append(v)
        accs.append(a)
        jerks.append(j_dec)
        times.append(t)

    # t1 ~ t2
    a1 = accs[-1]
    v1 = vels[-1]
    x1 = dists[-1]

    for t in np.linspace(0.0, t2, 100, endpoint=True):

        a = a1
        v = v1 + (a1 * t)
        x = x1 + (v1 * t) + (0.5 * a1 * t * t)
        # print("t: %f, t_total: %f, x: %f, v: %f, a: %f" % (t, t + t1, x, v, a))

        dists.append(x)
        vels.append(v)
        accs.append(a)
        jerks.append(0.0)
        times.append(t + t1)

    # t2 ~ t3
    a2 = accs[-1]
    v2 = vels[-1]
    x2 = dists[-1]

    for t in np.linspace(0.0, t3, 100, endpoint=True):

        a = a2 + (j_acc * t)
        v = v2 + (a2 * t) + (0.5 * j_acc * t * t)
        x = x2 + (v2 * t) + (0.5 * a2 * t * t) + ((1.0/6.0) * j_acc * t * t * t)
        # print("t: %f, t_total: %f, x: %f, v: %f, a: %f" % (t, t + t1 + t2, x, v, a))

        dists.append(x)
        vels.append(v)
        accs.append(a)
        jerks.append(j_acc)
        times.append(t + t1 + t2)

    return dists, vels, accs, jerks, times


def triangle_shape_v1(v0, ved,  a0, a_min, j_acc, j_dec):

    a_min2_square = 2 * (ved - v0 + (0.5 * a0 * a0 / j_dec)) * ((j_dec * j_acc) / (j_acc - j_dec))
    a_min2 = -np.sqrt(a_min2_square)

    t1 = (a_min2 - a0) / j_dec
    if t1 < 0.01:
        t1 = 0.0
    print("t1: %f" % t1)

    dists = []
    vels = []
    accs = []
    jerks = []
    times = []

    t2 = 0.0
    a1 = 0.0
    v1 = 0.0
    x1 = 0.0
    if t1 > 1e-3:
        print("v1")
        t2 = -a_min2 / j_acc
        if t2 < 0.01:
            t2 = 0.0 
        print("t2: %f" % t2)

        # 0 ~ t1
        for t in np.linspace(0.0, t1, 100, endpoint=True):

            a = a0 + (j_dec * t)
            v = v0 + (a0 * t) + (0.5 * j_dec * t * t)
            x = (v0 * t) + (0.5 * a0 * t * t) + ((1.0 / 6.0) * j_dec * t * t * t)
            # print("t: %f, t_total: %f, x: %f, v: %f, a: %f" % (t, t, x, v, a))

            dists.append(x)
            vels.append(v)
            accs.append(a)
            jerks.append(j_dec)
            times.append(t)

            a1 = accs[-1]
            v1 = vels[-1]
            x1 = dists[-1]

    else:
        print("v2")
        t2 = -a0 / j_acc
        if t2 < 0.01:
            t2 = 0.0
        print("t2: %f" % t2)

        a1 = a0
        v1 = v0

        v_true = (0.5) * j_acc * t2 * t2
        if math.fabs(v_true - v0) > 0.03:
            print("invalid v0 setting: v_true: %f, v0: %f" % ((v_true * 3.6), (v0 * 3.6)))
            return [0.0], [0.0], [0.0], [0.0], [0.0]

    # t1 ~ t2
    for t in np.linspace(0.0, t2, 100, endpoint=True):

        a = a1 + (j_acc * t)
        v = v1 + (a1 * t) + (0.5 * j_acc * t * t)
        x = x1 + (v1 * t) + (0.5 * a1 * t * t) + ((1.0/6.0) * j_acc * t * t * t)
        # print("t: %f, t_total: %f, x: %f, v: %f, a: %f" % (t, t + t1, x, v, a))

        dists.append(x)
        vels.append(v)
        accs.append(a)
        jerks.append(0.0)
        times.append(t + t1)

    return dists, vels, accs, jerks, times


def calc_stop_dist(v0, ved, a0, a_min, j_acc, j_dec):

    if a0 < a_min:
        print("invalid a0 setting: a0: %f, a_min: %f" % (a0, a_min))
        return [0.0], [0.0], [0.0], [0.0], [0.0]

    if v0 < ved:
        print("ved is larger than v0: v0: %f, ved: %f" % (v0, ved))
        return [0.0], [0.0], [0.0], [0.0], [0.0]

    t_min = (ved - v0 + (0.5 * a0 * a0 / j_dec) - (0.5 * a_min * a_min / j_dec) + (0.5 * a_min * a_min / j_acc)) / a_min
    # print("t min : %f" % t_min)

    dists = []
    vels = []
    accs = []
    jerks = []
    times = []

    if t_min > 0:
        print("type: trapezoid")
        dists, vels, accs, jerks, times = trapezoid_shape(v0, a0, a_min, j_acc, j_dec, t_min)
    else:
        print("type: triangle")
        dists, vels, accs, jerks, times = triangle_shape_v1(v0, ved, a0, a_min, j_acc, j_dec)

    return dists, vels, accs, jerks, times



In [3]:
@interact(v0_kmh=FloatSlider(min=0, max=20, step=1, value=10, continuous_update=False), # [km/h]
          ved_kmh=FloatSlider(min=0, max=20, step=1, value=0, continuous_update=False), # [km/h]
          a0=FloatSlider(min=-2.5, max=2.5, step=0.1, value=0.0, continuous_update=False), # [m/s]
          min_acc=FloatSlider(min=-2.5, max=0.0, step=0.1, value=-1.0, continuous_update=False), # [m/s^2]
          acc_jerk=FloatSlider(min=0.0, max=1.5, step=0.01, value=0.1, continuous_update=False), # [m/s^3]
          dec_jerk=FloatSlider(min=-1.5, max=0.0, step=0.01, value=-0.1, continuous_update=False), # [m/s^3]
         )
def visualize(v0_kmh, ved_kmh, a0, min_acc, acc_jerk, dec_jerk):

    dists = []
    vels = []
    accs = []
    jerks = []
    times = []

    dists, vels, accs, jerks, times = calc_stop_dist(v0_kmh / 3.6, ved_kmh / 3.6,  a0, min_acc, acc_jerk, dec_jerk)

    plt.figure(figsize=(20, 10))
    plt.subplot(2, 2, 1)
    plt.plot(times, dists)
    plt.xlabel("Time [sec]")
    plt.ylabel("Distance [m]")
    plt.title("Distance [m]")
    print("(x, t, v, a) = (%f, %f, %f, %f)" % (dists[-1], times[-1], vels[-1], accs[-1]))

    plt.subplot(2, 2, 2)
    vels_kmh = list(map(lambda x: x * 3.6, vels))
    plt.plot(times, vels_kmh)
    plt.xlabel("Time [sec]")
    plt.ylabel("Velocity [m/s]")
    plt.title("Velocity [m/s]")

    plt.subplot(2, 2, 3)
    plt.plot(times, accs)
    plt.xlabel("Time [sec]")
    plt.ylabel("Acceleration [m/s^2]")
    plt.title("Acceleration [m/s^2]")

    plt.subplot(2, 2, 4)
    plt.plot(times, jerks)
    plt.xlabel("Time [sec]")
    plt.ylabel("Jerk [m/s^3]")
    plt.title("Jerk [m/s^3]")



interactive(children=(FloatSlider(value=10.0, continuous_update=False, description='v0_kmh', max=20.0, step=1.…

In [4]:
# png_file: str
display_png(Image("./how_to_calc_dedist_with_acc_jerk_constraints.png"))

FileNotFoundError: No such file or directory: './how_to_calc_dedist_with_acc_jerk_constraints.png'

FileNotFoundError: No such file or directory: './how_to_calc_dedist_with_acc_jerk_constraints.png'